In [16]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Prepare training, validation and test data

In [3]:
def create_lstm_data(X, y, k):
    '''
    input:
        data - the numpy matrix of (n, p+1) shape, where n is the number of rows,
               p+1 is the number of predictors + 1 target column
        k    - the length of the sequence, namely, the number of previous rows 
               (including current) we want to use to predict the target.
    output:
        X_data - the predictors numpy matrix of (n-k, k, p) shape
        y_data - the target numpy array of (n-k, 1) shape
    '''
    # initialize zero matrix of (n-k, k, p) shape to store the n-k number
    # of sequences of k-length and zero array of (n-k, 1) to store targets
    X_data = np.zeros([X.shape[0]-k, k, X.shape[1]])
    y_data = []
    
    # run loop to slice k-number of previous rows as 1 sequence to predict
    # 1 target and save them to X_data matrix and y_data list
    for i in range(k, X.shape[0]):
        cur_sequence = X[i-k: i, :]
        
        X_data[i-k,:, :] = cur_sequence.reshape(1, k, X_data.shape[2])
        y_data.append(y[i-1])
    
    return X_data, np.asarray(y_data)

In [13]:
df_train = pd.read_csv("./data/preprocessed_data.csv")
df_test = pd.read_csv("./data/preprocessed_validation_data.csv")

# Concat with keys
temp = pd.get_dummies(pd.concat([df_train,df_test],keys=[0,1]), columns=['time_of_day', 'time_of_week', 'time_of_year'])

# Selecting data from multi index 
df_train, df_test = temp.xs(0),temp.xs(1)

scaler = MinMaxScaler()
scaling_columns = ['hydro', 'micro', 'thermal', 'wind', 'total', 'y', 'sys_reg', 'flow', 'y_prev']
df_train[scaling_columns] = scaler.fit_transform(df_train[scaling_columns])
df_test[scaling_columns] = scaler.transform(df_test[scaling_columns])

/var/folders/w2/4g3c4yrn38g2nwwdhjdlx6fc0000gn/T/ipykernel_47928/4061346065.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[scaling_columns] = scaler.fit_transform(df_train[scaling_columns])


In [14]:
y = df_train.pop("y").values
X = df_train.values.astype('float32')

no_rows = X.shape[0]
data_valid = X[int(no_rows*0.8):,:]
y_valid = y[int(no_rows*0.8):]
data_train = X[:int(no_rows*0.8),:]
y_train = y[:int(no_rows*0.8)]

# Setup and train model

In [ ]:
def LSTM_model(X_train, y_train, X_valid, y_valid, k):
    # Define and compile LSTM model
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(k, X_train.shape[2])))
    model.add(LSTM(50, return_sequences = False))
    #model.add(Dropout(0.2))
    #model.add(LSTM(50, return_sequences = True))
    #model.add(Dropout(0.2))
    #model.add(LSTM(50))
    #model.add(Dropout(0.2))
    model.add(Dense(1, "linear"))
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='mse')#,  metrics=[''])
    # Early stopping and best model checkpoint parameters
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max', verbose=2, patience=5)
    mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', mode='min', verbose=2, save_best_only=True)
    # Train the model 
    history = model.fit(X_train, y_train,
                        validation_data=(X_valid, y_valid),
                        epochs=5, verbose=2, callbacks=[es, mc])

In [11]:
seq_len = 10
 
X_train, y_train = create_lstm_data(data_train, y_train, seq_len)
X_valid, y_valid = create_lstm_data(data_valid, y_valid, seq_len)

LSTM_model(X_train, y_train, X_valid, y_valid, seq_len)

(218476, 51)

# Validate

In [18]:
seq_len = 10
model = keras.models.load_model('best_model.h5')
forecast_window_len = 24
start_ind = 70
model_input = X[start_ind:start_ind+seq_len]
forecasts = [y[start_ind+seq_len-1]]
for pred_no in range(0, forecast_window_len):
    print("pred_no: ", model_input)
    forecast = model(np.array([model_input]))
    cp = np.zeros((seq_len, 11))
    cp[0:seq_len-2] = model_input[1:seq_len-1]
    cp[-1] = model_input[seq_len-1]
    cp[-1, -1] = forecast
    model_input = cp
    forecasts.append(forecast)

pred_no:  [[0.39259163 0.2305624  0.33175355 0.15375513 0.36304292 0.8101266
  0.1768399  0.34806824 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.         1.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.39259163 0.2305624  0.33175355 0.15375513 0.36304292 0.8101266
  0.1768399  0.33528802 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.         1.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.   

ValueError: could not broadcast input array from shape (8,51) into shape (8,11)

In [ ]:
forecasts = np.array(forecasts)
forecasts = forecasts.reshape(forecast_window_len+1)

pyplot.plot(range(seq_len-1, seq_len+forecast_window_len-1), y[start_ind+seq_len-1:start_ind+seq_len+forecast_window_len-1], label='y_true')
pyplot.plot(range(seq_len-1, seq_len+forecast_window_len), forecasts, label='y_pred')
pyplot.plot(range(0, seq_len), y[start_ind:start_ind+seq_len], label='hist')

pyplot.legend()
pyplot.show()

# Test